* Квантитативный стафф: отношение аналитических форм будущего времени ко всем
* Наличие причастий настоящего времени как критерий НСВ (считай их отношение ко всем остальным временам?), отношение деепричастий СВ и НСВ между собой. СВ / НСВ (ибо НСВ больше)
* Отрицание перед глаголом
________
* Разметка окончаний и суффиксов причастий и дее-.
* Фильтрация плохо представленных глаголов без видовой оппозиции
* Финальный дроп датасета в джейсон.
________
* Замерять качество на каждом этапе.
* Обучить на контекстах руберт.
* На признаках кэтбуст.
________
* ??? Мб прототипический стафф. Обучить глов на Токенах СВ/НСВ вместо глаголов и прокинуть к получившимся эмбеддингам.
* 2 слоя с софтмаксом в конце.

# Importing packages / opening dataset

In [37]:
!pip install pandas==2.1.4

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import re
from itertools import zip_longest
import os
import json

import numpy as np
import pandas as pd

from IPython.display import display

In [40]:
def combine_json_files(folder_path):
  combined_data = pd.DataFrame()
  # not_biaspectuals_or_not_parsed = []
  for filename in os.listdir(folder_path):
      if not filename.endswith('both.json') and filename.endswith('.json'):
        lemma, aspect_json = filename.split("_")
        aspect = aspect_json[:-5]
        file_path = os.path.join(folder_path, filename)
        if filename[:-5].endswith('imp'):
          imp_df = pd.read_json(file_path, orient='records')
        elif filename[:-5].endswith('perf'):
          perf_df = pd.read_json(file_path, orient='records')

          if imp_df.shape[1] == 0 or perf_df.shape[1] == 0:
            pass
            # imp_df.drop(imp_df, inplace=True, axis=1)
            # perf_df.drop(perf_df, inplace=True, axis=1)
            # not_biaspectuals_or_not_parsed.append(lemma)
          else:
            try:
              combined_data = pd.concat([combined_data, imp_df, perf_df])
            except UnicodeDecodeError:
              continue
  return combined_data #, not_biaspectuals_or_not_parsed

def add_morphonology(df, data_path):
  morphon_data = pd.read_json(data_path, orient='records')
  return pd.merge(df, morphon_data, on='лемма', how='inner')

folder_path = '/content/drive/MyDrive/biaspectual_verbs'
morphonology_data_path = '/content/drive/MyDrive/morphonology_dataset.json'
df = combine_json_files(folder_path)
df = add_morphonology(df, morphonology_data_path)
print(df.shape)
df.head(1)

(31958, 11)


,основной анализ,словоформа,лемма,контекст,грамматика,семантика,похожие слова,синтаксические свойства слова,доп. признаки,ударение,морфемный разбор
0,омонимия снята автоматически,бюрократизирует,бюрократизировать,"ремесленниками в своей профессии, малыми частн...","глагол, 3-е лицо, действительный, изъявительно...",расплють (0.39892),NaN,бюрократизировать,"глагол, 3-е лицо, действительный, будущее, изъ...",бю-ро-кра-ти-зи́-ро-вать,Корень: -бюрократ-; интерфиксы: -из-ир-; суффи...


In [41]:
print(df.shape)
print(df.isna().sum())
df['контекст'].dropna(inplace=True)

(31958, 11)
основной анализ                     41
словоформа                           0
лемма                                0
контекст                          1152
грамматика                           0
семантика                           94
похожие слова                    13425
синтаксические свойства слова        0
доп. признаки                      680
ударение                             0
морфемный разбор                     0
dtype: int64


In [42]:
df['ударение'] = df['ударение'].apply(lambda stress: ''.join(re.split(r'[-·]', stress)))
df['ударный интерфикс -и́р-'] = df['ударение'].apply(
    lambda stress: bool(re.search(r'и́ровать', stress)))
df['ударный -а- в суффиксе -ова́-'] = df['ударение'].apply(
    lambda stress: bool(re.search(r'ова́ть', stress)))
df['опущение/отсутствие -ова- в словоформе'] = df['словоформа'].apply(
    lambda word: bool(re.search(r'ова', word)))

print(df['ударный -а- в суффиксе -ова́-'].sum(),
      df['ударный интерфикс -и́р-'].sum())

6997 22349


In [43]:
# добавь ударные гласные в объединении в начале
stressed_a = df['ударение'].apply(lambda x: a[0] if (a := re.search(r'\w*изова́ть(ся)?', '')) else '').unique()
nonstressed_a = df['ударение'].apply(lambda x: a[0] if (a := re.search(r'\w*(?<!из)ова́ть(ся)?', '')) else '').unique()
print(f'{stressed_a},\n{nonstressed_a}')

[''],
['']


In [44]:
stressed_iz = df['ударение'].apply(lambda x: a[0] if (a := re.search(r'\w*и́зовать(ся)?', x)) else '').unique()
nonstressed_iz = df['ударение'].apply(lambda x: a[0] if (a := re.search(r'\w*изовать(ся)?', x)) else '').unique()
print(f'{stressed_iz},\n{nonstressed_iz}')

[''],
['']


In [45]:
stressed_ir = df['ударение'].apply(lambda x: a[0] if (a := re.search(r'\w+и́ровать(ся)*', x)) else '').unique()
nonstressed_ir = df['ударение'].apply(lambda x: a[0] if (a := re.search(r'\w*ирова́ть(ся)*', x)) else '').unique()
nonstressed_ir2 = df['ударение'].apply(lambda x: a[0] if (a := re.search(r'\w*ировать(ся)*', x)) else '').unique()
print(f'{stressed_ir},\n{nonstressed_ir},\n{nonstressed_ir2}')

['бюрократизи́ровать' 'вакууми́ровать' 'вакцини́ровать' '' 'визи́ровать'
 'вулканизи́ровать' 'газифици́ровать' 'вульгаризи́ровать'
 'гаранти́роваться' 'ганти́ровать' 'гаранти́ровать' 'германизи́ровать'
 'гарни́ровать' 'гектографи́ровать' 'гомогенизи́ровать'
 'госпитализи́ровать' 'градуи́ровать' 'гудрони́ровать' 'дати́ровать'
 'девальви́ровать' 'дегенери́ровать' 'дегази́ровать' 'дегради́ровать'
 'дезерти́ровать' 'дезавуи́ровать' 'дезактиви́ровать' 'дезинформи́ровать'
 'дезориенти́ровать' 'декласси́роваться' 'декоди́ровать' 'деклари́ровать'
 'декори́ровать' 'демаски́ровать' 'демократизи́ровать' 'делеги́ровать'
 'денатури́ровать' 'денационализи́ровать' 'депорти́ровать'
 'дестабилизи́ровать' 'деформи́роваться' 'дешифри́ровать'
 'диагности́ровать' 'дислоци́ровать' 'диспони́ровать' 'дози́ровать'
 'дифференци́ровать' 'дифференци́роваться' 'дроссели́ровать'
 'дрени́ровать' 'документи́ровать' 'европеизи́роваться' 'дубли́роваться'
 'дубли́ровать' 'идентифици́ровать' 'инвентаризи́ровать' 'инверти

In [46]:
stress_vars = []
stress_vars.extend(stressed_a)
stress_vars.extend(nonstressed_a)
stress_vars.extend(stressed_iz)
stress_vars.extend(nonstressed_iz)
stress_vars.extend(stressed_ir)
stress_vars.extend(nonstressed_ir)
stress_vars.extend(nonstressed_ir2)
set(df['ударение']) ^ set(filter(bool, stress_vars))

{'авторизова́ть',
 'авторизова́ться',
 'адресова́ть',
 'адресова́ться',
 'аккредитова́ться',
 'акцентова́ть',
 'акцептова́ть',
 'арендова́ть',
 'ассигнова́ть',
 'атакова́ть',
 'аттестова́ть',
 'бежа́ть (дореформ. бѣжа́ть)',
 'венча́ть',
 'вокализова́ть',
 'гази́рова́ть',
 'гармонизова́ть',
 'дарова́ть',
 'дезорганизова́ть',
 'демобилизова́ться',
 'денатурализова́ться',
 'децентрализова́ть',
 'дешифрова́ть',
 'досле́довать',
 'жени́ть',
 'иллюминова́ть',
 'иммобилизова́ть',
 'инвентаризова́ть',
 'инструментова́ть',
 'ионизова́ть',
 'испо́льзовать',
 'испове́доваться',
 'колонизова́ть',
 'компьютеризова́ть',
 'контрассигнова́ть',
 'контратакова́ть',
 'конфирмова́ть',
 'конфискова́ть',
 'крести́ть',
 'крести́ться',
 'легализова́ть',
 'линчева́ть',
 'локализова́ть',
 'локализова́ться',
 'материализова́ть',
 'милитаризова́ть',
 'минова́ть',
 'мо́лвить',
 'модернизова́ть',
 'насле́довать',
 'натурализова́ть',
 'натурализова́ться',
 'нейтрализова́ть',
 'норми́рова́ть',
 'обеща́ть',
 'обеща́ть

In [47]:
from itertools import chain

df['приставка 1'] = df['морфемный разбор'].apply(
    lambda word: morpheme[0] if (
        morpheme := re.search(r'(?<=приставка:\s)\w+(?=-;)',
                              word, flags=re.IGNORECASE)) else '')
df['корень 1'] = df['морфемный разбор'].apply(
    lambda word: morpheme[0] if (
        morpheme := re.search(r'(?<=корень:\s-)\w+(?=-;)',
                              word, flags=re.IGNORECASE)) else '')

df['морфемный разбор'] = df['морфемный разбор'].apply(
     lambda word: dict(morpheme.lower().strip().split(': ') for morpheme in morphemes)
     if (morphemes := word.split('; ')) else '')

In [48]:
# pd.json_normalize(df['морфемный разбор'])

In [49]:
interfixes = ['интерфикс 1', 'интерфикс 2']
suffixes = ['суффикс 1', 'суффикс 2']

df_interfixes = pd.json_normalize(df['морфемный разбор'].apply(
     lambda analysis: dict(zip(interfixes, chain.from_iterable(
         analysis.get(string, '').split('-')[1:-1] for string in
         analysis.keys() if 'интерфикс' in string)))))

df_suffixes = pd.json_normalize(df['морфемный разбор'].apply(
     lambda analysis: dict(zip(suffixes, chain.from_iterable(
         analysis.get(string, '').split('-')[1:] for string in
         analysis.keys() if 'суффикс' in string)))))

In [50]:
df = df.merge(df_interfixes, left_index=True, right_index=True)
df = df.merge(df_suffixes, left_index=True, right_index=True)

In [51]:
df['интерфикс 1'].fillna('', inplace=True)
df['интерфикс 2'].fillna('', inplace=True)
df['суффикс 1'].fillna('', inplace=True)
df['суффикс 2'].fillna('', inplace=True)

mask = (df['суффикс 1'] == 'ирова?')
df.loc[mask, 'интерфикс 1'] = 'ир'
df.loc[mask, 'суффикс 1'] = 'ова'

print(f'''Приставки:
  1) {df['приставка 1'].unique()}''')
print(f'''Корни:
  1) {df['корень 1'].unique()}''')
print(f'''Интерфиксы:
  1) {df['интерфикс 1'].unique()},
  2) {df['интерфикс 2'].unique()}''')
print(f'''Суффиксы:
  1) {df['суффикс 1'].unique()},
  2) {df['суффикс 2'].unique()}''')

Приставки:
  1) ['' 'де' 'дез' 'до' 'им' 'интер' 'ин' 'ис' 'контр' 'о' 'пере' 'раз' 'рас'
 'ре' 'со' 'экс' 'экстра' 'э' 'аб' 'аг' 'ан']
Корни:
  1) ['бюрократ' 'вакуум' 'вакцин' 'вен' 'вокал' 'виз' 'вулкан' 'газ' 'вульгар'
 'гармон' 'гарант' 'гант' 'герман' 'гарнир' 'гект' 'гом' 'глаз' 'госпитал'
 'гофр' 'грав' 'граду' 'дар' 'гудрон' 'дат' 'вальв' 'генер' 'деград'
 'дезертир' 'дезаву' 'актив' 'информ' 'орган' 'ориент' 'класс' 'код'
 'деклар' 'декор' 'маск' 'мобил' 'дем' 'делег' 'натур' 'наци' 'депорт'
 'стабил' 'форм' 'центр' 'шифр' 'диагност' 'дислоц' 'диспон' 'доз'
 'дифференц' 'след' 'дроссел' 'дрен' 'документ' 'европ' 'дубл' 'жен'
 'иллюмин' 'идент' 'инвентар' 'инверт' 'инвест' 'индустри' 'иници'
 'инкасс' 'инкуб' 'инкримин' 'инкруст' 'инкорпор' 'интерпелл' 'инструмент'
 'пол' 'тон' 'интерфер' 'фильтр' 'инфиц' 'инъец' 'ион' 'йот' 'исповед'
 'польз' 'кальк' 'кальц' 'канон' 'капитул' 'капт' 'карбон' 'карт' 'касс'
 'кастр' 'катализ' 'каталог' 'катапульт' 'квал' 'колон' 'компенс'
 'ком

In [52]:
not_empty_prefix = df[df['приставка 1'].str.strip() != '']
not_empty_2_interfix = df[df['интерфикс 2'].str.strip() != '']
not_empty_2_suffix = df[df['суффикс 2'].str.strip() != '']

not_empty_prefix['лемма'].unique(), not_empty_2_interfix['лемма'].unique(), not_empty_2_suffix['лемма'].unique()

(array(['девальвировать', 'дегенерировать', 'дегазировать',
        'дезактивировать', 'дезинформировать', 'дезорганизовать',
        'дезориентировать', 'деклассироваться', 'декодировать',
        'демаскировать', 'демобилизоваться', 'денатурализоваться',
        'денатурировать', 'денационализировать', 'дестабилизировать',
        'деформироваться', 'децентрализовать', 'дешифрировать',
        'дешифровать', 'доследовать', 'иммобилизовать', 'интерполировать',
        'интонировать', 'инфильтрировать', 'использовать',
        'контратаковать', 'опробовать', 'опосредовать',
        'переадминистрировать', 'передислоцировать', 'перекодировать',
        'переориентировать', 'переориентироваться', 'переформулировать',
        'перефразировать', 'разгерметизировать', 'разгерметизироваться',
        'разминировать', 'расконсервировать', 'регрессировать',
        'репатриировать', 'реэвакуировать', 'сопутствовать',
        'эксгумировать', 'экспортировать', 'экстраполировать',
        'эмигр

In [53]:
lemma_wordform = np.stack([df['лемма'], df['словоформа'].str.lower()], axis=1).tolist()
df['словоформа - стемма'] = [pair[1].replace(pseudo_ending, '') if (pseudo_ending := os.path.commonprefix(pair)) else '' for pair in lemma_wordform]
df['словоформа - стемма'] =  df['словоформа - стемма'].apply(lambda x: 'ть' if not x else x)

part_gerund_suf = ['енн', 'ущ', 'ющ', 'ащ', 'ящ', 'ом', 'ем', 'им', 'вш', 'ши', 'нн', 'т', 'а', 'я', 'в', 'ш']
suf_4 = ['н', 'в', 'щ', 'ш']

df['суффикс 3'] = df['словоформа - стемма'].apply(lambda x: next((x[:len(suf)] for suf in part_gerund_suf if x.startswith(suf)), '').strip())
df['словоформа - стемма'] =  df['словоформа - стемма'].apply(lambda x: next((x[len(suf):] for suf in part_gerund_suf if x.startswith(suf)), '').strip())
df['окончание'] =  df['словоформа - стемма'].apply(lambda x: x if not x.endswith('ся') or not x.endswith('сь') else x[:-2])
df.drop('словоформа - стемма', axis=1, inplace=True)

# в перспективе можно выделить суффиксы дее-/причастий, постфиксы, флексию, обучить модель на морфемные разборы.
print(df['суффикс 3'].unique())
print(df['окончание'].unique())

['' 'т' 'нн' 'вш' 'в' 'я' 'ш' 'а' 'ющ' 'ем' 'енн' 'ящ' 'ом']
['' 'ь' 'ых' 'ую' 'ой' 'ые' 'ого' 'ыми' 'ым' 'ому' 'ый' 'ая' 'ом' 'ое'
 'аго' 'его' 'ий' 'ыя' 'ою' 'ейся' 'ей' 'ем' 'ие' 'ему' 'их' 'им' 'ими'
 'ись' 'ийся' 'емуся' 'ихся' 'ее' 'ееся' 'иеся' 'т' 'ымъ' 'ыхъ' 'омъ' 'ъ'
 'тируются' 'ься' 'имся' 'уюся' 'егося' 'е' 'а' 'емся' 'аяся' 'улирует'
 'имися' 'о']


In [54]:
df['похожие слова'] = df.apply(lambda row: row['семантика'] if isinstance(row['семантика'], str) and re.search(r'\(0.\d+\)', row['семантика']) else row['похожие слова'], axis=1)
df['семантика'].fillna('', inplace=True)
df['семантика'] = df['семантика'].apply(lambda x: '' if re.search(r'\(0.\d+\)', x) else x)
df['синтаксические свойства слова'] = df.apply(lambda row: '' if row['синтаксические свойства слова'] == row['лемма'] else row['синтаксические свойства слова'], axis=1)
df['доп. признаки'] = df.apply(lambda row: '' if isinstance(row['доп. признаки'], str) and row['доп. признаки'].startswith('глагол, ') else row['доп. признаки'], axis=1)
df['похожие слова'].fillna('', inplace=True)

In [55]:
prefix_pattern = r'{prefix}{root}\w*овывать'

imp_pairs = df.apply(lambda row: re.search(
    prefix_pattern.format(prefix=row['приставка 1'], root=row['корень 1']),
    row['похожие слова']).group() if
                      re.search(prefix_pattern.format(prefix=row['приставка 1'],
                                                      root=row['корень 1']),
                                row['похожие слова']) else '', axis=1)
imp_pairs.unique()

array(['', 'конфисковывать', 'организовывать', 'реализовывать',
       'ассигновывать'], dtype=object)

In [56]:
prefix_pattern = r'(про|за|с|со|от|ото|на|по|вы|о|раз|разо|рас|об|обо|у){root}\w+ть'

perf_pairs = df.apply(lambda row: re.search(
    prefix_pattern.format(root=row['корень 1']),
    row['похожие слова']).group() if
                      re.search(prefix_pattern.format(root=row['корень 1']),
                                row['похожие слова']) else '', axis=1)
perf_pairs.unique()

array(['', 'завизировать', 'награвировать', 'отградуировать',
       'закодировать', 'продекларировать', 'замаскировать',
       'расшифровывать', 'расшифровать', 'задокументировать', 'поженить',
       'проинформировать', 'спользовать', 'скомпенсировать',
       'помассировать', 'омеблировать', 'заминировать', 'промолвить',
       'пообещать', 'спробовать', 'опосредствовать', 'сорганизовать',
       'запатентовать', 'сформулировать', 'отпрепарировать',
       'разгерметизировать', 'законсервировать', 'прогрессировать',
       'порекомендовать', 'отретировать', 'отсалютовать', 'особоровать',
       'застенографировать', 'простерилизовать', 'протелеграфировать',
       'растиражировать', 'зафиксировать', 'заарендовать',
       'заасфальтировать', 'забетонировать'], dtype=object)

In [57]:
df['имеет коррелят СВ'] = perf_pairs.str.strip() != ''
df['имеет коррелят НСВ'] = imp_pairs.str.strip() != ''

Grammar, semantics, syntactic and additional features processing

In [58]:
df['грамматика'] = df['грамматика'].fillna('').apply(lambda x: '' if x is None else x)
df['семантика'] = df['семантика'].fillna('').apply(lambda x: '' if x is None else x)
df['синтаксические свойства слова'] = df['синтаксические свойства слова'].fillna('').apply(lambda x: '' if x is None else x)
df['доп. признаки'] = df['доп. признаки'].fillna('').apply(lambda x: '' if x is None else x)

In [59]:
grammar_properties = {
    'лицо': {'1-е лицо', '2-е лицо', '3-е лицо'},
    'число': {'единственное', 'множественное'},
    'вид': {'совершенный', 'несовершенный'},
    'наклонение / форма': {'изъявительное', 'повелительное', 'повелительное 2', 'инфинитив', 'деепричастие', 'причастие'},
    'залог': {'действительный', 'страдательный', 'медиальный'},
    'время': {'настоящее', 'будущее', 'прошедшее'},
    'число': {'единственное', 'множественное'},
    'падеж': {'именительный', 'родительный', 'дательный', 'винительный', 'творительный', 'предложный', 'родительный 2', 'винительный 2', 'предложный 2', 'звательный', 'счётная форма'},
    'одушевленность': {'одушевленное', 'неодушевленное'},
    'полнота формы': {'полная форма', 'краткая форма'},
    'род': {'мужской', 'женский', 'средний', 'общий'},
    'переходность': {'переходный', 'непереходный'}
}

syntactic_properties = {
    'аргументы': {'подлежащее (субъект)', 'дополнение (второй аргумент)', 'дативно-инструментальное дополнение', 'предложное дополнение', 'инфинитивное подлежащее', 'клаузальный комплемент', 'инфинитивный комплемент'},
    'модификаторы': {'наречный модификатор', 'обстоятельственная клауза', 'количественный модификатор', 'атрибутивный модификатор', 'атрибутивная клауза', 'релятивная клауза', 'местоименный атрибутивный модификатор', 'аппозитив (приложение)', 'другие именные модификаторы'},
    'не-зависимости': {'сочиненный элемент', 'второй элемент паратаксиса', 'вынос темы', 'элемент неоднословного оборота', 'элемент списка', 'второй элемент в имени', 'второй элемент неграмматической структуры', 'компаунд', 'прочие «плоские» отношения', 'самоисправление', 'ошибка раздельного написания'},
    'позиция клаузы в иерархии': {'главная клауза', 'подчиненная клауза'},
    'тип клаузы': {'клауза с эллипсисом', 'глагольная клауза', 'именная клауза', 'предикативная клауза'},
    'оборот': {'распространенный причастный оборот', 'деепричастный оборот'},
    'вершина': {'вершина предложения'},
    'есть зависимые': {'есть зависимые'}
}

semantics = {
    'таксономия': {'движение', 'помещение объекта', 'изменение состояния или признака', 'бытийная сфера', 'местонахождение', 'контакт и опора', 'посессивная сфера', 'ментальная сфера', 'восприятие', 'поведение человека', 'физиологическая сфера', 'природное явленин', 'звук', 'свет', 'запах'},
    'таксономия конкретнее': {'изменение положения тела, части тела', 'создание физического объекта', 'уничтожение', 'существование', 'начало существования', 'прекращение существования', 'положение тела в пространстве', 'эмоция', 'воля', 'речь'},
    'каузация': {'каузативные глаголы', 'некаузативные глаголы'},
    'словообразование 1': {'отыменные', 'отглагольные', 'отадъективные', 'непроизводные глаголы'},
    'словообразование 2': {'приставочные глаголы', 'семельфактивы', 'вторичные имперфективы (-ива-, -ва-, -а-)'},
    'служебные глаголы': {'фазовые', 'служебные каузативные'}
}

# слово перед/после: любого знака препинания, точки, запятой, двоеточия, точки с запятой, тире, дефиса, восклицательного знака, вопросительного знака
additional_features = {
    'токен перед': {'bmark', 'bdot', 'bcomma', 'bcolon', 'bsemicolon', 'bdash', 'bhyphen', 'bexcl', 'bques'},
    'токен после': {'amark', 'amark', 'adot', 'acomma', 'acolon', 'asemicolon', 'adash', 'ahyphen', 'aexcl', 'aques'},
    'прочее': {'norm', 'bastard', 'first', 'last'},
    'начинается с заглавной': {'capital'},
    'повтор': {'lexred', 'posred', 'casered', 'numred', 'timered', 'genderred', 'persred', 'animred'},
    'тип оборота 1': {'mw:ADV', 'mw:ADVPRO', 'mw:CONJ', 'mw:PARENTH', 'mw:PART', 'mw:PR', 'mw:PRAEDIC', 'mw:SPRO'},
    'тип оборота 2': {'mw:degr:max', 'mw:dir', 'mw:dist:max', 'mw:dur:max', 'mw:ev', 'mw:ev:neg', 'mw:ev:posit', 'mw:loc:body'},
    'тип оборота 3': {'mw:place', 'mw:rep', 'mw:shift', 'mw:space', 'mw:speed:max', 'mw:time'}
}

In [60]:
def invert_dict(input_dict):
  empty_placeholder = tuple()
  inverted_dict = {}
  for k, v in input_dict.items():
    inverted_dict.update(dict(zip_longest(v, empty_placeholder, fillvalue=k)))
  return inverted_dict

grammar_properties_inv = invert_dict(grammar_properties)
semantics_inv = invert_dict(semantics)
syntactic_properties_inv = invert_dict(syntactic_properties)
additional_features_inv = invert_dict(additional_features)

In [61]:
df['основной анализ'].value_counts()

,count
основной анализ,
омонимия снята автоматически,28340
омонимия снята,3577


In [62]:
for property_ in grammar_properties:
    df[property_] = df['грамматика'].apply(
    lambda verb_properties: ' / '.join(
        [gp for gram_prop in verb_properties.split(', ')
        if grammar_properties_inv.get(gp := gram_prop.strip()) == property_]
        )
    )

for property_ in semantics:
    df[property_] = df['семантика'].apply(
    lambda verb_properties: ''.join(
        [gp for gram_prop in verb_properties.split(', ')
        if semantics_inv.get(gp := gram_prop.strip()) == property_]
        )
    )


for property_ in syntactic_properties:
    df[property_] = df['синтаксические свойства слова'].apply(
    lambda verb_properties: ' / '.join(
        [gp for gram_prop in verb_properties.split(', ')
        if syntactic_properties_inv.get(gp := gram_prop.strip()) == property_]
        )
    )

for property_ in additional_features:
    df[property_] = df['доп. признаки'].apply(
    lambda verb_properties: ' / '.join(
        [gp for gram_prop in verb_properties.split(', ')
        if additional_features_inv.get(gp := gram_prop.strip()) == property_]
        )
    )

In [63]:
df.drop(['основной анализ', 'грамматика', 'семантика', 'похожие слова', 'синтаксические свойства слова', 'доп. признаки', 'ударение', 'морфемный разбор'], inplace=True, axis=1)
df['возвратность'] = df['лемма'].apply(lambda row: row.endswith('ся'))
df.dropna(inplace=True)

In [64]:
for column in grammar_properties:
  print(f'{column}: {df[column].unique()}')

лицо: ['3-е лицо' '' '1-е лицо' '2-е лицо']
число: ['единственное' '' 'множественное']
вид: ['несовершенный' 'совершенный']
наклонение / форма: ['изъявительное' 'деепричастие' 'причастие' 'инфинитив' 'повелительное']
залог: ['действительный' 'страдательный' 'медиальный'
 'действительный / медиальный']
время: ['настоящее' 'прошедшее' '' 'будущее']
падеж: ['' 'родительный' 'винительный' 'предложный' 'именительный' 'творительный'
 'дательный']
одушевленность: ['' 'неодушевленное' 'одушевленное']
полнота формы: ['' 'полная форма' 'краткая форма']
род: ['' 'женский' 'мужской' 'средний']
переходность: ['переходный' 'непереходный' 'непереходный / переходный' '']


In [65]:
df.drop(['прочее', 'тип оборота 1', 'тип оборота 2', 'тип оборота 3'], axis=1, inplace=True)
df['повтор'] = df['повтор'] != ''
df['токен перед'] = df['токен перед'] != ''
df['токен после'] = df['токен после'] != ''
df['начинается с заглавной'] = df['начинается с заглавной'] != ''
try:
  for column in additional_features:
    print(f'{column}: {df[column].unique()}')
except KeyError as e:
  print(f'No column {e}')

токен перед: [False  True]
токен после: [False  True]
No column 'прочее'


In [66]:
for column in syntactic_properties:
  print(f'{column}: {df[column].unique()}')

аргументы: ['' 'клаузальный комплемент' 'инфинитивный комплемент'
 'инфинитивное подлежащее' 'подлежащее (субъект)'
 'дативно-инструментальное дополнение']
модификаторы: ['' 'обстоятельственная клауза' 'атрибутивная клауза'
 'другие именные модификаторы' 'атрибутивный модификатор'
 'релятивная клауза' 'аппозитив (приложение)' 'наречный модификатор']
не-зависимости: ['' 'сочиненный элемент' 'второй элемент паратаксиса'
 'элемент неоднословного оборота' 'прочие «плоские» отношения']
позиция клаузы в иерархии: ['' 'подчиненная клауза' 'главная клауза']
тип клаузы: ['' 'глагольная клауза' 'именная клауза'
 'предикативная клауза / предикативная клауза'
 'именная клауза / глагольная клауза' 'предикативная клауза'
 'глагольная клауза / именная клауза']
оборот: ['' 'деепричастный оборот' 'распространенный причастный оборот'
 'распространенный причастный оборот / распространенный причастный оборот'
 'распространенный причастный оборот / деепричастный оборот']
вершина: ['' 'вершина предложения']

In [67]:
df.drop(['служебные глаголы'], axis=1, inplace=True)
try:
  for column in semantics:
    print(f'{column}: {df[column].unique()}')
except KeyError as e:
  print(f'No column {e}')

таксономия: ['' 'посессивная сфера' 'изменение состояния или признака'
 'местонахождение' 'движение' 'помещение объекта' 'ментальная сфера']
таксономия конкретнее: ['' 'создание физического объекта' 'речь' 'прекращение существования'
 'начало существования' 'существование']
каузация: ['' 'каузативные глаголы' 'некаузативные глаголы']
словообразование 1: ['' 'отыменные' 'отадъективные' 'отглагольные' 'непроизводные глаголы']
словообразование 2: ['' 'приставочные глаголы']
No column 'служебные глаголы'


In [68]:
grouped = df.groupby('лемма')['вид'].nunique()
lemmas_to_keep = grouped[grouped > 1].index
df = df[df['лемма'].isin(lemmas_to_keep)]
print(df['вид'].value_counts())
print(df['лемма'].nunique())
df['лемма'].value_counts().tail(10)

вид
несовершенный    15996
совершенный       8288
Name: count, dtype: int64
217


,count
лемма,
педализировать,6
алгоритмизировать,6
фаворизировать,5
интерпеллировать,5
дросселировать,5
авторизоваться,4
ректифицировать,4
алкоголизировать,4
пульверизировать,4


In [69]:
df.isna().sum().sum()

0

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24284 entries, 0 to 31956
Data columns (total 45 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   словоформа                              24284 non-null  object
 1   лемма                                   24284 non-null  object
 2   контекст                                24284 non-null  object
 3   ударный интерфикс -и́р-                 24284 non-null  bool  
 4   ударный -а- в суффиксе -ова́-           24284 non-null  bool  
 5   опущение/отсутствие -ова- в словоформе  24284 non-null  bool  
 6   приставка 1                             24284 non-null  object
 7   корень 1                                24284 non-null  object
 8   интерфикс 1                             24284 non-null  object
 9   интерфикс 2                             24284 non-null  object
 10  суффикс 1                               24284 non-null  object
 11  суффикс

In [71]:
df.describe()

,словоформа,лемма,контекст,ударный интерфикс -и́р-,ударный -а- в суффиксе -ова́-,опущение/отсутствие -ова- в словоформе,приставка 1,корень 1,интерфикс 1,интерфикс 2,...,позиция клаузы в иерархии,тип клаузы,оборот,вершина,есть зависимые,токен перед,токен после,начинается с заглавной,повтор,возвратность
count,24284,24284,24284,24284,24284,24284,24284,24284,24284,24284,...,24284,24284,24284,24284,24284,24284,24284,24284,24284,24284
unique,4617,217,18548,2,2,2,16,199,8,4,...,3,7,5,2,2,2,2,2,2,2
top,,соборовать,,True,False,True,,мин,ир,,...,,,,,,False,False,False,False,False
freq,858,407,80,17113,19105,15279,21894,780,13926,20682,...,21833,20444,23003,23443,20675,23628,23202,24044,21911,23022


In [74]:
df.to_json('/content/drive/MyDrive/biaspectual_verbs_preprocessed.json', orient='records')